In [52]:
import pandas as pd
import ast
import numpy as np
from src.Classifier import Classifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
from TextUtils import clean_article_url
from flatten_dict import flatten

In [53]:
#Read corpus generated by the Corpus class
corpus_df = pd.read_csv('../data/corpus/corpus_augmented_backtranslation.csv')

In [54]:
#For evaluation of the classifier read manually labeled data
label = pd.read_csv('../data/labelledarticles/LabeledArticles.csv',delimiter=';')
label['formats/url'] = label['formats/url'].apply(lambda x: clean_article_url(x))
labels_dict = label[['formats/url','expected_product_hierarchy_aggr']].to_dict(orient='records')
label_values = [{dic['formats/url']: dic['expected_product_hierarchy_aggr']} for dic in labels_dict]
label_lookup = {}
for d in label_values:
    label_lookup.update(d)

In [45]:
grouped_corpus_df = corpus_df.groupby('group_key').agg(descriptor_text=('feature_descriptor', ', '.join))
grouped_corpus_df['y_true']=grouped_corpus_df.index
grouped_corpus_df = grouped_corpus_df.replace({'y_true': label_lookup})
grouped_corpus_df['y_true'] = grouped_corpus_df['y_true'].fillna('Unlabeled')
grouped_corpus_df['y_true'] = grouped_corpus_df['y_true'].astype(str)
#Encode writtern labels to ids
le = preprocessing.LabelEncoder()
le.fit(grouped_corpus_df['y_true'].values)
#Apply encoding to labels
grouped_corpus_df['true']=le.transform(grouped_corpus_df['y_true'])

In [57]:
len(grouped_corpus_df['y_true'].unique())

20

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
X_train, X_test, y_train, y_test = train_test_split(
            grouped_corpus_df['descriptor_text'], grouped_corpus_df['y_true'], test_size = 0.5, random_state = 0)
clf_dummy = DummyClassifier(random_state=42, strategy="stratified")
#establishing random_state for reproducibility
clf_dummy.fit(X_train, y_train)
y_pred = clf_dummy.predict(X_test)
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

               Unlabeled       0.20      0.09      0.13        11
                     cms       0.00      0.00      0.00         0
          commerce-cloud       0.00      0.00      0.00         3
         community-cloud       0.00      0.00      0.00         2
    consumer-goods-cloud       0.00      0.00      0.00         2
           crm-analytics       0.00      0.00      0.00         3
financial-services-cloud       0.00      0.00      0.00         0
            health-cloud       0.00      0.00      0.00         1
                  heroku       0.00      0.00      0.00         1
             integration       0.00      0.00      0.00         1
     manufacturing-cloud       0.00      0.00      0.00         2
         marketing-cloud       0.18      0.38      0.24         8
             mytrailhead       0.00      0.00      0.00         1
                platform       0.00      0.00      0.00         6
         

/Users/mzavarzin/PyCharmProjects/Clean/ASFTE/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mzavarzin/PyCharmProjects/Clean/ASFTE/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
